# Seminar 1 - A Song of Graphs and Search

---

**Course**: Graphs and Network Analysis

**Degree**: Artificial Intelligence Degree (UAB)

**Topic**: Practical seminar that includes exercises from units 1 to 6

**Activity description**: Most of us are familiar with the Game of Thrones books or series. For those who do not know it, it is a fictional series from the HBO chain, inspired by the series of novels "A Song of Ice and Fire", which tells the experiences of a group of characters from different noble houses on the fictional continent of *Westeros* to have control of the Iron Throne and rule the seven kingdoms that make up the territory. The series' success has spawned many blogs and other sources about the series, with additional resources. The graphs that we propose to use in this exercise represent the characters of the series (or books) as nodes, and their co-appearance in a scene (the weights of the edges are higher if two characters appear simultaneously more times). So we have a social network of characters. We will use these graphs to work on some of the concepts seen in the first units of the course (graph and node metrics, search and routes). Finally, synthetic graphs that simulate a realistic network will be generated.

## Qualification

**Submission**: An '.ipynb' file from the colab corresponding to each group will be delivered (this very same file, adding the code blocks and explanations that correspond to each activity). To get the file you will need to go to File --> Download. Remember that you will have to answer and analyze the different problems. Coding alone will NOT be evaluated: explaining and reasoning about the solution of the problem is essential. **You should provide explanations of the obtained results for at least the exercises marked with the 💬 symbol**.
The outcome of this seminar will thus be an analysis of the network at different levels: global metrics, node importance, shortest paths, random graphs, and visualization.

**Delivery form**: The work must be done in **groups of two people** and delivered through the virtual campus (in the section corresponding to Seminar 1).

**Doubts**: For any questions, apart from class sessions, you can contact cristina.perez@uab.cat.

**Deadline**: March 13th (during all day).

**Marks**: The grade of the seminars (seminar 1 + seminar 2) has a weight of 10% on the final grade of the subject.


# Authors

**Lab group:** GrupLab-9

**Student 1 - Name (NIU): 1668936**

**Student 2 - Name (NIU): 1672981**

## 1. Environment setup
----

The main libraries that will be used in this seminar are the following:

* [NetworkX](https://networkx.github.io/)
* [Pandas](https://pandas.pydata.org/)
* [Matplotlib](https://matplotlib.org/)
* [NumPy](https://numpy.org/)



In [ ]:
!pip install --upgrade scipy networkx

In [ ]:
!apt-get install -y graphviz libgraphviz-dev pkg-config

In [ ]:
!pip install pygraphviz

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

## 2. Data collection

---

This seminar is based on data from *Game of Thrones* and "A Song of Ice and Fire" curated by Andrew Beveridge. Data is available from two different github repositories:

* [Book to Network](https://github.com/mathbeveridge/asoiaf)
* [Script to Network](https://github.com/mathbeveridge/gameofthrones)

In each of them, there is a *data* folder with several *.csv* files that encode nodes and edges of different networks.

To download the data in the *colab* environment you can run the following command:

```
$ !wget https://raw.githubusercontent.com/mathbeveridge/repo_name/master/data/file_id-nodes.csv
$ !wget https://raw.githubusercontent.com/mathbeveridge/repo_name/master/data/file_id-edges.csv
```


where,

* **repo_name** is the name of the repository, *asoiaf* for the Books and *gameofthrones* for the Script.
* **file_id** is the ID of the file you can find with the link. This indicates the book or season number.

For example, to download the graph of the first season of the series, we would run:

```
$ !wget https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-nodes.csv
$ !wget https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-edges.csv
```

The downloaded files can be found in */content/file_name*.

For this activity, we will work with the graph generated from all the books.


*  **Download the two .csv files corresponding to the graph generated from all the books (asoiaf-all)**.

In [ ]:
!wget https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-nodes.csv
!wget https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-edges.csv

## 3. Data load

---

The function *csv_to_graph()* creates a NetworkX graph from the *.csv* files encoding edges and nodes.

In [ ]:
def csv_to_graph(file_id_nodes: str, file_id_edges: str, origin: str = 'book') \
                    -> nx.graph:
    """Return a nx.graph

    Build a graph given a csv file for nodes and edge.
    origin controls the source of the graph to adapt the node features.
    """

    if origin == 'book':
        key1, key2 = 'weight', 'book'
    elif origin == 'script':
        key1, key2 = 'Weight', 'Season'
    else:
        raise NameError('Unknown origin {}'.format(origin))

    nodes = pd.read_csv(file_id_nodes)
    edges = pd.read_csv(file_id_edges)

    if key2 not in edges:
        key2 = 'id'

    g = nx.Graph()
    for row in nodes.iterrows():
        g.add_node(row[1]['Id'], name=row[1]['Label'])

    for row in edges.iterrows():
        g.add_edge(row[1]['Source'],row[1]['Target'],
                   weight=1/row[1][key1], id=row[1][key2])

    return g


* **Create a NetworkX graph from the downloaded files using the `csv_to_graph` function.** [Optionally, you can repeat the process with the graph generated from the series]

In [ ]:
g_book = csv_to_graph('asoiaf-all-nodes.csv', 'asoiaf-all-edges.csv', origin='book')

* **Generate a first exploratory visualization of the graph.**

In [ ]:
plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['figure.dpi'] = 100

pos = graphviz_layout(g_book, prog="neato")
nx.draw(g_book, pos, with_labels=False, arrows=False, node_size=30)
plt.show()

## 4. General graph metrics
---

Perform a general summary of the Network properties.

* **💬  Obtain the order, size and density of the graph, as well as the average degree of its nodes.**


In [ ]:
graph_order = g_book.order()
graph_size = g_book.size()
graph_density = nx.density(g_book)
average_degree = np.mean(list(dict(g_book.degree()).values()))
print('Order:', graph_order)
print('Size:', graph_size)
print('Density:', graph_density)
print('Average degree:', average_degree)

💬 :

* **Check that it is a connected undirected graph.**

In [ ]:
# check that is a connected undirected graph
print('Is connected:', nx.is_connected(g_book))
print('Is undirected:', not nx.is_directed(g_book))

* **💬 Make a small report on the metrics of the given graph (diameter, radius, average network distance, clustering coefficient).**

In [ ]:
print("diameter:", nx.diameter(g_book))
print("radius:", nx.radius(g_book))
print("average network distance:", nx.average_shortest_path_length(g_book))
print("average clustering coefficient:", nx.average_clustering(g_book))

💬 :

## 5. Centrality metrics: Characters' importance
---


In this section, we will study the importance of the characters according to their centrality in the graph.

* **Compute the 10 most central nodes in the network taking into account the different types of centrality (degree, betweenness, closeness and eigenvector centrality). Moreover, use page rank to assess importance of the characters.**

  * *centrality_bar_plot()*: Given the corresponding centrality draw a bar graph.
  * 💬 Try to reason about the changes that occur with the different types of centrality.

In [ ]:
import matplotlib.pyplot as plt
def centrality_bar_plot(centrality, name='betweenness', n=10):
    # take the top n nodes by quantity of centrality
    centrality = dict(Counter(centrality).most_common(n))
    values = list(centrality.values())
    label = list(centrality.keys())

    df = pd.DataFrame({'Name': label, name: values})
    ax = df.plot.bar(x='Name', y=name, rot=90)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 4]

degree_centrality = nx.degree_centrality(g_book) # Degree Centrality
betweenness_centrality = nx.betweenness_centrality(g_book) # Betweenness Centrality
closeness_centrality = nx.closeness_centrality(g_book) # Closeness Centrality
eigen_centrality = nx.eigenvector_centrality(g_book) # Eigenvector Centrality


centrality_bar_plot(degree_centrality, name='degree')
centrality_bar_plot(betweenness_centrality, name='betweenness')
centrality_bar_plot(closeness_centrality, name='closeness')
centrality_bar_plot(eigen_centrality, name='eigen')

plt.rcParams['figure.figsize'] = [12, 12]

In [ ]:
# Page rank to assess the importance of the characters
page_rank = nx.pagerank(g_book)
centrality_bar_plot(page_rank, name='page_rank')

plt.show()

💬 : In the closeness centrality there doesn't seem to be a significant difference out of the top 10 nodes. The measure shows when a node is near the center of the network, so all other nodes have a very similar value because they're really close to the center.

In the betweenness centrality bar plot, we can visualize a gradual difference in value for the nodes. We start with a node that has a really high value (the center) and clearly the node is relevant in the information flow since it's the one that brokers between groups the most. As we reach for the neighbors of this node, this value decreases because we're incrementing the distance between the highest node and the others.

As for the degree centrality, the plot shows also a gradual decrease, which is normal because all 10 nodes don't hold the same importance in the graph as we see that their connections decrease as we go through each node. This is an usual pattern in the context of social networks when the node that has the most followers is the most important while the rest have more or less an average number of followers which is usually much more lower.

In the eigenvector centrality, we see the same pattern, the latter nodes are connected to less important nodes and have a lower degree centrality, while the highest are connected to more influential nodes. As we go through the nodes we notice a decrease in this number.

In page rank, we have more or less the same as in the degree centrality.

* **What is the subgraph generated by the best connected characters?**
  * Use closeness centrality to generate the graph of the 25 most central characters.

In [ ]:
def centrality_subgraph(g, centrality, name='closeness', n=25):
    centrality = dict(Counter(centrality).most_common(n))
    nodes = list(centrality.keys())
    sg = g.subgraph(nodes)
    plt.title('Top {} nodes by {}'.format(n, name))
    pos = graphviz_layout(sg, prog="neato")
    nx.draw(sg, pos, with_labels=True, arrows=False, node_size=30)
    plt.show()
    return sg

In [ ]:
g_subgraph = centrality_subgraph(g_book, closeness_centrality, name='closeness_centrality', n=25)
print(g_subgraph)

* **Draw this subgraph where the nodes are of size proportional to their centrality. Mark the most central and the less central node in the graph (for instance, use the color of the node to highlight it).**
  * Use *closeness centrality* and scale it appropriately to emphasize the importance of different nodes.

In [ ]:
def centrality_subgraph(g, centrality, name='closeness', n=25):
    centrality = dict(Counter(centrality).most_common(n))
    nodes = list(centrality.keys())
    sg = g.subgraph(nodes)

    node_sizes = [v * 5000 for v in centrality.values()]

    most_central_node = max(centrality, key=centrality.get)
    least_central_node = min(centrality, key=centrality.get)

    plt.title('Top {} nodes by {}'.format(n, name))
    pos = nx.spring_layout(sg)
    nx.draw(sg, pos, with_labels=True, arrows=False, node_size=node_sizes, cmap=plt.cm.Blues)
    nx.draw_networkx_nodes(sg, pos, nodelist=[most_central_node], node_color='red', node_size=node_sizes[0])
    nx.draw_networkx_nodes(sg, pos, nodelist=[least_central_node], node_color='green', node_size=node_sizes[-1])

    plt.show()
    return sg

g_subgraph = centrality_subgraph(g_book, closeness_centrality, name='closeness_centrality', n=25)

* **Draw the tree that the BFS and DFS algorithm would generate to traverse the graph starting from the least central node of the network according to *closeness centrality*.**
  * Use *closeness centrality* and scale it appropriately to emphasize the importance of different nodes.
  * To get the positions of the nodes, you can use the `graphviz_layout(tree, prog='dot')` command.
  * 💬 Comment on the obtained result.


In [ ]:
least_central_node = min(closeness_centrality, key=closeness_centrality.get)

bfs_tree = nx.bfs_tree(g_book, least_central_node)

dfs_tree = nx.dfs_tree(g_book, least_central_node)

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.title('BFS Tree')
bfs_pos = graphviz_layout(bfs_tree, prog='dot')
nx.draw(bfs_tree, pos=bfs_pos, with_labels=False, node_size=250, node_color='red')

plt.subplot(122)
plt.title('DFS Tree')
dfs_pos = graphviz_layout(dfs_tree, prog='dot')
nx.draw(dfs_tree, pos=dfs_pos, with_labels=False, node_size=250, node_color='green')

plt.show()

💬 : The BFS tree indeed goes through the nodes by levels as seen in the plot while the DFS tree goes through a branch.

* **💬 Compute the shortest path between the least and the most central nodes in the complete graph.**

In [ ]:
least_central_node = min(closeness_centrality, key=closeness_centrality.get)

most_central_node = max(closeness_centrality, key=closeness_centrality.get)

shortest_path = nx.shortest_path(g_book, source=least_central_node, target=most_central_node)

print("Shortest path between the least and the most central nodes:", shortest_path)

💬 :

## 6. Random graph models
----
Up to this point, we have worked with a graph generated from the data extracted from the *Song of Ice and Fire* books. In the real world, however, obtaining the data needed to construct this graph can become very complex and expensive. This is one of the reasons why, over time, the synthetic generation of graphs has been studied.

In this section we will work on the different models described in class. We will generate random graphs and study their properties.

* **Generate random graphs with the Uniform, Gilbert and Barabási-Albert models. Fix the number of nodes to the order of the studied graph. Adjust the rest of the parameters of the graph generation function to obtain graphs with similar number of edges.**

In [ ]:
nedges = g_book.number_of_edges()
nnodes = g_book.number_of_nodes()
seed = 42
print(nnodes, nedges)

### Erdös-Rény: Uniform Model (gnm)

In [ ]:
g_uniform = nx.gnm_random_graph(nnodes, nedges, seed=seed)
print(g_uniform.number_of_nodes(), g_uniform.number_of_edges())

### Erdös-Rény: Gilbert Model (gnp)


In [ ]:
total_edges = nnodes*(nnodes-1)/2
g_gilbert = nx.gnp_random_graph(nnodes, p=nedges/total_edges, seed=seed)
print(g_gilbert.number_of_nodes(), g_gilbert.number_of_edges())

### Barabási-Albert Model



In [ ]:
# we compute m
g_barbasi = nx.barabasi_albert_graph(n=nnodes, m=4, seed=seed)
print(g_barbasi.number_of_nodes(), g_barbasi.number_of_edges())

In [ ]:
g_dict = {'Book': g_book, 'Uniform': g_uniform, 'Erdos': g_gilbert, 'Barbasi': g_barbasi}

* **💬 Show the order and size of the graph as well as the average degree and clustering coefficient of its nodes. Compute also the intervals between the maximum and minimum centralities for each family of synthetic graphs. Make a small report of the main metrics. Which random graph resembles more closely the graph from the books?**
     * You can set the graph generation using a random seed. This way, two different runs will generate exactly the same graph.

In [ ]:
for k, g in g_dict.items():
	print("------", k, "------")
	print("Order of: {}".format(g.order()))
	print("Size of: {}".format(g.size()))
	print("Average degree of: {}".format(np.mean(list(dict(g.degree()).values()))))
	print("Average clustering coefficient: {}".format(nx.average_clustering(g)))

In [ ]:
for k, g in g_dict.items():
	deg_centrality = nx.degree_centrality(g)
	max_dc = max(deg_centrality, key=deg_centrality.get)
	min_dc = min(deg_centrality, key=deg_centrality.get)
	bet_centrality = nx.betweenness_centrality(g)
	max_bc = max(bet_centrality, key=bet_centrality.get)
	min_bc = min(bet_centrality, key=bet_centrality.get)
	cl_centrality = nx.closeness_centrality(g)
	max_clc = max(cl_centrality, key=cl_centrality.get)
	min_clc = min(cl_centrality, key=cl_centrality.get)
	eig_centrality = nx.eigenvector_centrality(g)
	max_ec = max(eig_centrality, key=eig_centrality.get)
	min_ec = min(eig_centrality, key=eig_centrality.get)

	print("------", k, "------")
	print("Interval of degree centrality: [{}, {}]".format(deg_centrality[min_dc], deg_centrality[max_dc]))
	print("Interval of betweenness centrality: [{}, {}]".format(bet_centrality[min_bc], bet_centrality[max_bc]))
	print("Interval of closeness centrality: [{}, {}]".format(cl_centrality[min_clc], cl_centrality[max_clc]))
	print("Interval of eigenvector centrality: [{}, {}]".format(eig_centrality[min_ec], eig_centrality[max_ec]))

💬 : The random grah that reassembles the most the book graph is the **Barbasi**. Overall, it is the one whose metrics approximate more the book graph, with some exceptions like the minimum degree centrality, which others approximate more.

* **💬 Check whether the networks (the three randomly generated ones and the network extracted from the books) follow a Power Law.**

In [ ]:
plt.rcParams['figure.figsize'] = [13, 10]

# check power law
fig, ax = plt.subplots(4, 1)
for i, (k, g) in enumerate(g_dict.items()):
	degree_sequence = sorted([d for n, d in g.degree()], reverse=True)
	degreeCount = {i: degree_sequence.count(i) for i in degree_sequence}
	deg, count = zip(*degreeCount.items())
	ax[i].bar(deg, count, width=0.80, color='b')
	ax[i].set_title(k)
	ax[i].set_xlabel('Degree')
	ax[i].set_ylabel('Count')
plt.tight_layout()
plt.show()

💬 : Only the Barbasi random graph follows a Power-law distribution, like the original graph from the book.